In [9]:
import rasterio
import geopandas as gpd
from scipy import spatial
import shapely
import pandas as pd

ImportError: libkea.so.1.4.7: cannot open shared object file: No such file or directory

In [ ]:
data = rasterio.open('../data/s2sr_30mpp0000000000-0000006912.tif'')
band1 = data.read(1)
height = band1.shape[0]
width = band1.shape[1]
cols, rows = np.meshgrid(np.arange(width), np.arange(height))
xs, ys = rasterio.transform.xy(data.transform, rows, cols)
lons= np.array(xs)
lats = np.array(ys)
print('lons shape', lons.shape)
lons_flat = lons.reshape(-1,1).flatten()
lats_flat = lats.reshape(-1,1).flatten()
coordinates = list(zip(lons_flat, lats_flat))
boundingBox = shapely.geometry.box(*data.bounds)
df = pd.read_csv('../data/iSDA_soil_data_filter.csv')
lat = df['longitude']
long = df['latitude']
geometry = [shapely.geometry.Point(xy) for xy in zip(lat, long)]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=4326)
df2 = pd.DataFrame(columns=['geometry'])
df2.loc[0] = boundingBox
gdf2 = gpd.GeoDataFrame(df2, geometry='geometry', crs=4326)
gdf_filtered = gdf[gdf.geometry.within(boundingBox)]
for idx, p in gdf_filtered.iterrows():
    # print(p.geometry.coords[0])
    mindist = spatial.distance.cdist(coordinates, [p.geometry.coords[0]], 'euclidean')
    idx_flat = np.argmin(mindist)
    idx_lons, idx_lats = int(idx_flat/lons.shape[1]), idx_flat%lons.shape[1]
    # print(idx, idx_lons, idx_lats)
    gdf_filtered.loc[idx, 'lons_250m'] = idx_lons
    gdf_filtered.loc[idx, 'lats_250m'] = idx_lats